# Автоматичне опрацювання української мови

## 1. Сегментація тексту

Прочитаймо текст "Тигроловів":

In [ ]:
with open("data/tyhrolovy.txt", "r") as f:
    tyhrolovy_raw = f.read()

print("У тексті є {} символів.".format(len(tyhrolovy_raw)))

In [ ]:
print(tyhrolovy_raw[:200])

**Сирий текст** варто сегментувати на абзаци, речення, слова. Для цього використаємо бібліотеку [tokenize_uk](https://github.com/lang-uk/tokenize-uk).

Як ви гадаєте:
- які бувають складнощі автоматичного розбиття тексту на речення?
- які бувають складнощі автоматичного розбиття тексту на слова?
- які бувають складнощі автоматичного розбиття тексту на абзаци?

In [ ]:
import tokenize_uk

In [ ]:
tyhrolovy_words = tokenize_uk.tokenize_words(tyhrolovy_raw)

print("У тексті є {} слів.".format(len(tyhrolovy_words)))
print()
print(tyhrolovy_words[:40])

In [ ]:
tyhrolovy_tokenized = tokenize_uk.tokenize_text(tyhrolovy_raw)

for i in tyhrolovy_tokenized[:8]:
    print(i)

## 2. Частиномовний аналіз

Поговорімо про складнощі автоматичного частиномовного аналізу:
- які слова можуть позначати різні частини мови?
- які частини мови може позначати словоформа *"край"*? *"прав"*? *"багатій"*?
- чи є неоднозначність у фразі *"коло друзів та незнайомців"*?

Більше прикладів:

<img src="img/ambiguous-pos.png" alt="Приклади слів, що позначають кілька частин мови" width="40%" height="40%" align="left"/>

Бібліотека [pymorphy2](https://github.com/kmike/pymorphy2) надає всі можливі варіанти розбору кожного слова, проте не визначає правильний розбір у контексті.

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [ ]:
morph.parse("край")

In [ ]:
morph.parse("прав")

In [ ]:
morph.parse("багатій")

In [ ]:
# Випробуйте інші слова. Гляньте на аналіз багатозначних слів ("до", "того") та нестандартних слів ("євробляха").

#### Як із цим працювати?

In [ ]:
morph.parse("багатій")[0]

In [ ]:
morph.parse("багатій")[0].normal_form

In [ ]:
morph.parse("багатій")[0].tag

In [ ]:
morph.parse("багатій")[0].tag.POS

In [ ]:
morph.parse("багатій")[0].tag.gender

In [ ]:
morph.parse("багатій")[0].tag.animacy

In [ ]:
morph.parse("багатій")[3].tag.animacy

Погляньмо на леми першого речення "Тигроловів":

In [ ]:
for word in tyhrolovy_words[9:32]:
    parsed_word = morph.parse(word)[0]
    print("{:10}\t{}".format(word, parsed_word.normal_form))

In [ ]:
for word in tyhrolovy_words[9:32]:
    parsed_word = morph.parse(word)[0]
    print("{:10}\t{:10}\t{}".format(word, parsed_word.normal_form, parsed_word.tag.POS))

## 3. Частотний аналіз тексту

#### Проведімо аналіз усіх слів у творах *"Тигролови"* та *"Собор"*:

In [ ]:
with open("data/sobor.txt", "r") as f:
    sobor_raw = f.read()
    sobor_words = tokenize_uk.tokenize_words(sobor_raw)

print("У тексті є {} слів.".format(len(sobor_words)))

In [ ]:
corpus_t = [morph.parse(word)[0] for word in tyhrolovy_words]

In [ ]:
corpus_s = [morph.parse(word)[0] for word in sobor_words]

#### Як знайти найчастотніші слова в романі?

In [ ]:
from collections import Counter

def most_freq(corpus, n):
    "Return the most common words."
    words = Counter()
    for word in corpus:
        words[word.word] += 1
    return words.most_common(n)

In [ ]:
most_freq(corpus_t, 20)

Як можна покращити функцію **most_freq**?

In [ ]:
def most_freq(corpus, n):
    "Return the most common lemmas."
    words = Counter()
    for word in corpus:
        if word.tag.POS in {"NOUN", "VERB", "ADJF"}:
            words[word.normal_form] += 1
    return words.most_common(n)

In [ ]:
most_freq(corpus_t, 20)

In [ ]:
COMMONLY_CONFUSED = {'бути', 'до', 'йога', 'мен', 'перед', 'про', 'тога'}

def most_freq(corpus, n):
    "Return the most common lemmas."
    words = Counter()
    for word in corpus:
        if word.tag.POS in {"NOUN", "VERB", "ADJF"} and word.normal_form not in COMMONLY_CONFUSED:
            words[word.normal_form] += 1
    return words.most_common(n)

In [ ]:
most_freq(corpus_t, 20)

In [ ]:
most_freq(corpus_s, 20)

#### Як знайти найчастотніші колокації в романі?

In [ ]:
def most_freq_colloc(corpus, n):
    "Return the most common adjective-noun collocations."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_colloc(corpus_t, 20)

In [ ]:
most_freq_colloc(corpus_s, 20)

In [ ]:
def most_freq_colloc_for_noun(corpus, n, noun):
    "Return the most common adjective-noun collocations with a specified noun."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.normal_form == noun and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_colloc_for_noun(corpus_t, 20, "ніч")

In [ ]:
most_freq_colloc_for_noun(corpus_s, 20, "ніч")

In [ ]:
# Спробуйте інші іменники, прикметники чи контексти.

In [ ]:
def most_freq_colloc_for_adj(corpus, n, adj):
    "Return the most common adjective-noun collocations with a specified adjective."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and prev_word.normal_form == adj:
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_colloc_for_adj(corpus_t, 20, "золотий")

In [ ]:
most_freq_colloc_for_adj(corpus_s, 20, "золотий")

In [ ]:
def most_freq_colloc_anim(corpus, n):
    "Return the most common adjective-noun collocations with animate nouns."
    collocs = Counter()
    for i in range(1, len(corpus)):
        word = corpus[i]
        prev_word = corpus[i-1]
        if word.tag.POS == "NOUN" and word.tag.animacy == "anim" and prev_word.tag.POS == "ADJF":
            collocs[(prev_word.normal_form, word.normal_form)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_colloc_anim(corpus_t, 20)

In [ ]:
most_freq_colloc_anim(corpus_s, 20)

In [ ]:
# Порівняйте колокації для іменників жіночого роду з колокаціями для іменників чоловічого роду.

In [ ]:
def most_freq_prep_adj_noun(corpus, n):
    "Return the most common prep-adj-noun collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "NOUN" and corpus[i-1].tag.POS == "ADJF" and corpus[i-2].tag.POS == "PREP":
            collocs[(corpus[i-2].word, corpus[i-1].word, word.word)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_prep_adj_noun(corpus_t, 20)

In [ ]:
most_freq_prep_adj_noun(corpus_s, 20)

In [ ]:
def most_freq_verb_prep_noun(corpus, n):
    "Return the most common verb-prep-noun collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "NOUN" and corpus[i-1].tag.POS == "PREP" and corpus[i-2].tag.POS == "VERB":
            collocs[(corpus[i-2].normal_form, corpus[i-1].word, word.word)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_verb_prep_noun(corpus_t, 20)

In [ ]:
most_freq_verb_prep_noun(corpus_s, 20)

In [ ]:
def most_freq_conj_adj(corpus, n):
    "Return the most common adj-and-adj collocations."
    collocs = Counter()
    for i in range(2, len(corpus)):
        word = corpus[i]
        if word.tag.POS == "ADJF" and corpus[i-1].word == "і" and corpus[i-2].tag.POS == "ADJF":
            collocs[(corpus[i-2].normal_form, corpus[i-1].word, word.normal_form)] += 1
    return collocs.most_common(n)

In [ ]:
most_freq_conj_adj(corpus_t, 20)

In [ ]:
most_freq_conj_adj(corpus_s, 20)

#### Як дослідити контексти слова?

In [ ]:
def show_concordance(corpus, lemma, n):
    contexts = []
    for i in range(len(corpus)):
        if corpus[i].normal_form == lemma:
            left_context = corpus[max(0, i-5) : i]
            right_context = corpus[i+1 : min(i+6, len(corpus))]
            left = " ".join([word.word for word in left_context])
            right = " ".join([word.word for word in right_context])
            contexts.append("{:30}\t{:10}\t{}".format(left, corpus[i].word, right))
    return contexts[:n]

In [ ]:
for i in show_concordance(corpus_t, "спати", 20):
    print(i)

In [ ]:
for i in show_concordance(corpus_s, "спати", 20):
    print(i)

## А як щодо інших текстів?

Скопіюйте текст будь-якої новини. Наприклад, https://hromadske.ua/posts/ukrayina-evakuyuye-z-uhanya-razom-zi-svoyimi-gromadyanami-do-25-inozemciv-moz.

In [ ]:
news = """
В Міністерстві охорони здоров’я повідомили, що Україна евакуює з карантинного міста Ухань китайської провінції Хубей 49 українців та до 25 іноземних громадян через смертельний коронавірус.
Про це на брифінгу розповів заступник міністра охорони здоров’я Віктор Ляшко.

«До 25 іноземних громадян. Хто ці особи вирішує Міністерство закордонних справ по дипломатичних каналах. Але вони усі будуть однаково проходити обсервацію, без різниці з якої країни», — зазначив чиновник.

Також він додав, що наразі МОЗ обрало два медзаклади, куди доправлять евакуйованих на 14-денний карантин.

«Сьогодні я їх не можу озвучити, але вони є, вони визначені і по них ведеться повна підготовка. Один основний, один резервний на випадок, якщо щось станеться під час моменту евакуації», — сказав Ляшко.

На брифінгу він також повідомив, що евакуацію українців з китайської провінції Хубей, яка є епіцентром спалаху коронавірусу, знову перенесли. Раніше очікувалося, що літак з українцями прибуде у середу 19 лютого, тепер його очікують у четвер, 20 лютого.

Раніше Міністерство закордонних справ Аргентини висловило подяку Україні за те, що українська влада погодилась разом зі своїми громадянами евакуювати й громадян Аргентини з китайського Уханя, епіцентру поширення коронавірусу. Скільки саме аргентинців забере український літак з Уханя, відомство не уточнило.

За останніми даними, кількість людей, інфікованих китайським коронавірусом, збільшилась до 71 331. Через вірус померли 1 775 пацієнтів. У Гонконгу зафіксували 57 випадків інфікування та один смертельний випадок.
"""

In [ ]:
corpus_news = [morph.parse(word)[0] for word in tokenize_uk.tokenize_words(news)]

In [ ]:
most_freq(corpus_news, 10)

In [ ]:
most_freq_colloc(corpus_news, 10)

#### Продовження заняття можна знайти за цим посиланням:
http://tiny.cc/0p63jz